<a href="https://colab.research.google.com/github/tjrgh822/class_itsystem/blob/master/rotate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Input

from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [0]:
train, test= mnist.load_data()
train_data, train_labels, test_data, test_labels = train[0], train[1], test[0], test[1]
import imutils

def rotate_image(data, rotate_class, filepath, filename1, filename2):
  data = data.reshape(-1, 28, 28)
  random_angle = np.random.choice(rotate_class,  data.shape[0])

  rotate_data = np.zeros(1, dtype=np.uint8)
  for i in range(len(data)):
    rotate_data = np.append(rotate_data, np.array(imutils.rotate(data[i], random_angle[i])))
    if i % 10000 == 0:
      print(i)
  rotate_data = np.delete(rotate_data, 0).reshape(-1, 28, 28, 1)
  np.save(f'{filepath}/{filename1}.npy', rotate_data)
  np.save(f'{filepath}/{filename2}.npy', random_angle)


rotate_image(test_data, range(-45, +45, 5), '/content/drive/My Drive', 'test_rotate_data+-45', 'test_rotate_labels+-45')



In [5]:
def onehotencoding(data, case):
  encoding = np.eye(len(case))[data]
  return encoding

train_x = np.load('/tf/class_itsystem/rotate_data+-45.npy')
train_y = np.load('/tf/class_itsystem/rotate_labels+-45.npy')
test_x = np.load('/tf/class_itsystem/test_rotate_data+-45.npy')
test_y = np.load('/tf/class_itsystem/test_rotate_labels+-45.npy')

train_y = onehotencoding(np.int32(train_y*0.2 + 9), list(range(-45, +45, 5)))
test_y = onehotencoding(np.int32(test_y*0.2 + 9), list(range(-45, +45, 5)))



datagen = ImageDataGenerator()


In [6]:
import tensorflow.keras.backend as K

def angle_difference(x, y):
    """
    Calculate minimum difference between two angles.
    """
    return 180 - abs(abs(x - y) - 180)


def angle_error(y_true, y_pred):
    """
    Calculate the mean diference between the true angles
    and the predicted angles. Each angle is represented
    as a binary vector.
    """
    diff = angle_difference(K.argmax(y_true), K.argmax(y_pred))
    return K.mean(K.cast(K.abs(diff), K.floatx()))


def angle_error_regression(y_true, y_pred):
    """
    Calculate the mean diference between the true angles
    and the predicted angles. Each angle is represented
    as a float number between 0 and 1.
    """
    return K.mean(angle_difference(y_true * 360, y_pred * 360))

'''    datagen = ImageDataGenerator( width_shift_range=5, 
                             height_shift_range=5, 
                             horizontal_flip=True, 
                             vertical_flip=True,
                             brightness_range = [0.2, 1.0],
                             zoom_range = [0.5, 1.0]
                             )'''


'    datagen = ImageDataGenerator( width_shift_range=5, \n                             height_shift_range=5, \n                             horizontal_flip=True, \n                             vertical_flip=True,\n                             brightness_range = [0.2, 1.0],\n                             zoom_range = [0.5, 1.0]\n                             )'

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
nb_filters = 64
pool_size = (2, 2)
kernel_size = (3, 3)
nb_classes = 18

# model definition
inputs = Input(shape=(34, 34, 1))
x = Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                  activation='relu')(inputs)
x = Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                  activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(nb_classes, activation='softmax')(x)

model = Model(inputs, x)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[angle_error])

model.summary()







Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 34, 34, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 11, 11, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320  

In [8]:
batch = 30
epochs = 10

# callbacks
checkpointer = ModelCheckpoint(
    filepath='model_1.h5',
    save_best_only=True
)
early_stopping = EarlyStopping(patience=30)

callbacks=[checkpointer, early_stopping]

model.fit(datagen.flow(train_x, train_y, batch_size=batch), 
          epochs = epochs,
          steps_per_epoch = train_x.shape[0]/ batch,
          validation_data = datagen.flow(test_x, test_y, batch_size= 1),
          validation_steps = train_y.shape[0], 
          verbose = 1
          )




Epoch 1/10
1994/2000 [============================>.] - ETA: 0s - loss: 2.8335 - angle_error: 4.5904

KeyboardInterrupt: 